## Introduction

ML algorithms require the input to be represented as a fixed-length feature vector.

When it comes to texts, one of the most common fixed-length features is bag-of-words.

One-hot encoding: huge vector, does not capture semantic

BOW features have two major weaknesses: 
 * they lose the ordering of the words
 * they also ignore semantics of the words
 
Paragraph Vector algorithm:
 * unsupervised
 * learns from variable-length pieces of texts (sentences, paragraphs, documents etc.)
 * generates fixed-length representation
 
 
Text classification and clustering tasks are implemented using logistic regression and k-means.
They require fixed length input.
Most common fixed-length vector representation for texts is the bag-of-words or bag-of-n-grams.

The word order is lost, and thus different sentences can have exactly the same representation, as long as the same words are used.

bag-of-n-grams considers the word order in short context but suffers from data sparsity and high dimensionality.

Bag-of-words and bag- of-n-grams have very little sense about the semantics of the words or more formally the distances between the words. This means that words “powerful,” “strong” and “Paris” are equally distant despite the fact that semantically, “power- ful” should be closer to “strong” than “Paris.”

While paragraph vectors are unique among paragraphs, the word vectors are shared. At prediction time, the paragraph vectors are inferred by fix- ing the word vectors and training the new paragraph vector until convergence.

The outcome is that after the model is trained, the word vectors are mapped into a vector space such that semantically similar words have similar vector representa- tions (e.g., “strong” is close to “powerful”).


## word2vec algorithm

We need a **vocabulary** that maps every word that appear in a document to an integer ID.

Every word is mapped to a unique vector, represented by a column in a matrix $W$. The column is indexed by the word ID  from the vocabulary. 

The concatenation or sum of the vectors is then used as features for prediction of the next word in a sentence.

<img src="images/word2vec.png" height="250" width="400"/> 

**What is the goal?**

The model is trained on a sequence of training words $w_1,w_2,...,w_T$. This is usually a large corpus of documents.

Given a **window** $[w_{t-k},...,w_{t-1}, w_{t+1},...,w_{t+k}]$, the model predicts the center word $w_t$. 

This is a **classification problem** where the classes are the words in the vocabulary. Each examples is a word window and the label is the center word.

More specifically, given a window, the model predicts the probability for each word $w_i$ in the vocabulary to be the center word: $p(w_i \mid w_{t-k},...,w_{t-1}, w_{t+1},...,w_{t+k})$

The goal of the model is to **maximize** the average probability:

$$
\frac{1}{T} \sum^{T-k}_{t=k} p(w_t \mid w_{t-k},...,w_{t-1}, w_{t+1},...,w_{t+k})
$$

Instead of the average probability we maximize the average log probability:

$$
\frac{1}{T} \sum^{T-k}_{t=k} log \ p(w_t \mid w_{t-k},...,w_{t-1}, w_{t+1},...,w_{t+k})
$$

This modification does not change the optimization objective because by maximizing a log probability we also maximize the probability. With this modification we can simply use standard cross entropy as loss function and minimize the loss using a gradient descent optimizer.

## How it works

Every word from the vocabulary is mapped to a unique vector, represented by a column in a matrix $W$.

Define a function $h(w_{t-k},...,w_{t-1}, w_{t+1},...,w_{t+k}; W)$ that takes a context as input and is parametrized by $W$. 

$h$ extracts the word vectors from $W$ and aggregates them by one of:
 * concatenation
 * average
 * sum

Than calculate:

$$
y = b + Uh(w_{t-k},...,w_{t-1}, w_{t+1},...,w_{t+k}; W)
$$

$y = [y_1,...,y_m]$ is a vector that has as many dimensions are we have words in the vocabulary. Each $y_i$ is the unnormalized probability that $w_i$ is the center word given the context.

$U$ is a (vocab_size, 2*k) weight matrix and $b$ is vocab_size dimensional bias vector.

We use the softmax function to normalize the probabilities:

$$
p(w_i \mid w_{t-k},...,w_{t-1}, w_{t+1},...,w_{t+k}) = \frac{e^{y_i}}{\sum_j {e^{y_j}}}
$$

Loss function:

**TODO**

$$
p(w_t \mid w_{t-k},...,w_{t-1}, w_{t+1},...,w_{t+k}) = \frac{e^{y_{w_t}}}{\sum_j {e^{y_j}}}
$$

$W$, $U$ and $b$ are the trainable weights of the model. Note that $W$ is a trainable parameter as well, its values are modified by the optimizer.

After training is done the columns in $W$ are the vector representations of the words in the vocabulary.

## Implementation

The implementation of the model is quite simple.

The function $h()$ can be implemented by two parts:
 1. An embedding layer provides the lookup of the column vectors in $W$
 2. Layers for concatenation, averaging or addition provide the aggregation

The calculation $y = b + Uh()$ is performed by a dense layer. The dense layer will be configured with a softmax activation that normalized the probabilities.

The loss will be calculated by a standard cross entropy function.
